<a href="https://colab.research.google.com/github/ZAHID188/Q-learning_collab/blob/main/Q_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import tensorflow as tf
print('Hi')

Hi
